In [64]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as opt
from tabulate import tabulate
from scipy.stats import laplace, uniform
import math

## Постановка задачи:
Сгенерировать выборку объёмом $100$ элементов для нормального распределения $N(x,0,1).$ 

По сгенерированной выборке оценить параметры $\mu$ и $\sigma$ нормального закона методом максимального правдоподобия. В качестве основной гипотезы $H_{0}$ будем считать, что сгенерированное распределение имеет вид $N(x,\hat{\mu}, \hat{\sigma})$. Проверить основную гипотезу, используя критерий согласия $\chi^{2}$. В качестве уровня значимости взять $\alpha = 0.05$. Привести таблицу вычислений $\chi^{2}$. 


Сгенерировать выборку объемом $20$ элементов для распределения Лапласа. Проверить гипотезу $H_0^*$ нормальности этой выборки, используя критерий согласия $\chi^{2}$.
	


In [98]:
points = 20
sample_size = 100
start, end = -1.5, 1.5
alpha = 0.05
p_ = 1 - alpha
k = 7

mu_, sigma_squared = 0, 1


def get_k(size):
    return math.ceil(1.72 * (size) ** (1/3))

## Метод максимального правдоподобия

In [81]:
def max_likelihood_estimation(sample):
    mu = np.mean(sample)
    sigma = np.std(sample)
    print("mu = ", np.around(mu, decimals=2),
          " sigma=", np.around(sigma, decimals=2))
    return mu, sigma

## Критерий согласия $\chi^{2}$

In [103]:
def calculate_chi2(p,n):
    tmp = np.multiply((n - sample_size * p), (n - sample_size * p))
    chi2 = np.divide(tmp, p * sample_size)
    return chi2

def is_hypo_accepted(quantile, chi2):
    if quantile > np.sum(chi2):
        return True
    return False

def find_all_probabilities(borders, hypothesis, sample):
    p = np.array(hypothesis(start))
    n = np.array(len(sample[sample < start]))

    for i in range(k - 2):
        p_i = hypothesis(borders[i + 1]) - hypothesis(borders[i])
        p = np.append(p, p_i)
        n_i = len(sample[(sample < borders[i + 1]) & (sample >= borders[i])])
        n = np.append(n, n_i)

    p = np.append(p, 1 - hypothesis(end))
    n = np.append(n, len(sample[sample >= end]))
    
    return p,n

    
def chi_square_criterion(sample, mu, sigma):
    hypothesis = lambda x: stats.norm.cdf(x, loc=mu, scale=sigma)
    borders = np.linspace(start, end, num=k - 1)
    p,n = find_all_probabilities(borders, hypothesis, sample)
    chi2 = calculate_chi2(p,n)
    quantile = stats.chi2.ppf(p_, k - 1)
    isAccepted = is_hypo_accepted(quantile, chi2)
    return chi2, isAccepted, borders, p, n

## Вывод результатов в таблицу

In [104]:
def build_table(chi2, borders, p, n, sample_size):
    rows = []
    headers = ["$i$", "$\\Delta_i = [a_{i-1}, a_i)$", "$n_i$", "$p_i$",
               "$np_i$", "$n_i - np_i$", "$(n_i - np_i)^2/np_i$"]   
    for i in range(0, len(n)):
        if i == 0:
            limits = ["$-\infty$", np.around(borders[0], decimals=2)]
        elif i == len(n) - 1:
            limits = [np.around(borders[-1], decimals=2), "$\infty$"]
        else:
            limits = [np.around(borders[i - 1], decimals=2), np.around(borders[i], decimals=2)]
        rows.append([i + 1, limits, n[i],
             np.around(p[i], decimals=4),
             np.around(p[i] * sample_size, decimals=2),
             np.around(p[i] - sample_size * p[i], decimals=2),
             np.around(chi2[i], decimals=2)] )
    rows.append(["\\sum", "--", np.sum(n), np.around(np.sum(p), decimals=4),
                 np.around(np.sum(p * sample_size), decimals=2),
                 -np.around(np.sum(n - sample_size * p), decimals=2),
                 np.around(np.sum(chi2), decimals=2)]
    )
    return tabulate(rows, headers, tablefmt="latex_raw")


In [105]:
k = get_k(100)
normal_s = np.random.normal(0, 1 , sample_size)
mu, sigma = max_likelihood_estimation(normal_s)
chi2, isAccepted, borders, p, n = chi_square_criterion(normal_s, mu, sigma)
print(build_table(chi2, borders, p, n, 100))
if isAccepted:
    print("Hypothesis Accepted!")
else:
    print("Hypothesis Not Accepted!")

mu =  0.03  sigma= 0.88
\begin{tabular}{llrrrrr}
\hline
 $i$   & $\Delta_i = [a_{i-1}, a_i)$   &   $n_i$ &   $p_i$ &   $np_i$ &   $n_i - np_i$ &   $(n_i - np_i)^2/np_i$ \\
\hline
 1     & ['$-\\infty$', -1.5]          &       2 &  0.0405 &     4.05 &          -4.01 &                    1.04 \\
 2     & [-1.5, -1.0]                  &      15 &  0.0794 &     7.94 &          -7.86 &                    6.27 \\
 3     & [-1.0, -0.5]                  &      11 &  0.1524 &    15.24 &         -15.08 &                    1.18 \\
 4     & [-0.5, 0.0]                   &      22 &  0.2132 &    21.32 &         -21.1  &                    0.02 \\
 5     & [0.0, 0.5]                    &      18 &  0.2175 &    21.75 &         -21.53 &                    0.65 \\
 6     & [0.5, 1.0]                    &      18 &  0.1619 &    16.19 &         -16.03 &                    0.2  \\
 7     & [1.0, 1.5]                    &      12 &  0.0879 &     8.79 &          -8.7  &                    1.18 \\
 8     & 

In [96]:
k = get_k(20)
normal_s = distribution = laplace.rvs(size=20, scale=1 / math.sqrt(2), loc=0)
mu, sigma = max_likelihood_estimation(normal_s)
chi2, isAccepted, borders, p, n = chi_square_criterion(normal_s, mu, sigma)
print(build_table(chi2, borders, p, n,20 ))
if isAccepted:
    print("Hypothesis Accepted!")
else:
    print("Hypothesis Not Accepted!")

mu =  0.12  sigma= 0.97
\begin{tabular}{llrrrrr}
\hline
 $i$   & $\Delta_i = [a_{i-1}, a_i)$   &   $n_i$ &   $p_i$ &   $np_i$ &   $n_i - np_i$ &   $(n_i - np_i)^2/np_i$ \\
\hline
 1     & ['$-\\infty$', -1.5]          &       0 &  0.047  &     0.94 &          -0.89 &                    0.69 \\
 2     & [-1.5, -0.5]                  &       4 &  0.2129 &     4.26 &          -4.05 &                    1.72 \\
 3     & [-0.5, 0.5]                   &      12 &  0.3909 &     7.82 &          -7.43 &                    2.7  \\
 4     & [0.5, 1.5]                    &       1 &  0.2712 &     5.42 &          -5.15 &                    2.04 \\
 5     & [1.5, '$\\infty$']            &       3 &  0.078  &     1.56 &          -1.48 &                    0.92 \\
 \sum  & --                            &      20 &  1      &    20    &           0    &                    8.06 \\
\hline
\end{tabular}
Hypothesis Accepted!


In [97]:
k = get_k(20)
normal_s = distribution =  uniform.rvs(size=20, loc=-math.sqrt(3), scale=2 * math.sqrt(3))
mu, sigma = max_likelihood_estimation(normal_s)
chi2, isAccepted, borders, p, n = chi_square_criterion(normal_s, mu, sigma)
print(build_table(chi2, borders, p, n, 20))
if isAccepted:
    print("Hypothesis Accepted!")
else:
    print("Hypothesis Not Accepted!")

mu =  -0.15  sigma= 0.77
\begin{tabular}{llrrrrr}
\hline
 $i$   & $\Delta_i = [a_{i-1}, a_i)$   &   $n_i$ &   $p_i$ &   $np_i$ &   $n_i - np_i$ &   $(n_i - np_i)^2/np_i$ \\
\hline
 1     & ['$-\\infty$', -1.5]          &       1 &  0.04   &     0.8  &          -0.76 &                    0.63 \\
 2     & [-1.5, -0.5]                  &       6 &  0.2841 &     5.68 &          -5.4  &                    2.11 \\
 3     & [-0.5, 0.5]                   &       8 &  0.4748 &     9.5  &          -9.02 &                    3.2  \\
 4     & [0.5, 1.5]                    &       5 &  0.1845 &     3.69 &          -3.51 &                    1.56 \\
 5     & [1.5, '$\\infty$']            &       0 &  0.0165 &     0.33 &          -0.31 &                    0.39 \\
 \sum  & --                            &      20 &  1      &    20    &           0    &                    7.88 \\
\hline
\end{tabular}
Hypothesis Accepted!
